In [1]:
%pip install chromadb langchain


[notice] A new release of pip available: 22.2.1 -> 23.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load data
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='./clubs.csv')
data = loader.load()

# # Comment out above AND Uncomment below to load url data
# %pip install unstructured
# from langchain.document_loaders import UnstructuredURLLoader
# urls = [
#     "https://www.berkeley.edu/campus-life/student-organizations/",
# ]
# loader = UnstructuredURLLoader(urls=urls)
# data = loader.load()

In [3]:
# Split texts
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

In [4]:
# Get llm and embeddings
%pip install gpt4all > /dev/null
%pip install sentence-transformers
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
local_path = (
    "./wizardlm-13b-v1.1-superhot-8k.ggmlv3.q4_0.bin" 
)
callbacks = [StreamingStdOutCallbackHandler()]
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# # Comment out above AND Uncomment below to use Open API Key
# %pip install openai tiktoken
# import os
# os.environ["OPENAI_API_KEY"] = "______"
# from langchain.embeddings.openai import OpenAIEmbeddings
# llm = OpenAI()
# embeddings = OpenAIEmbeddings()


[notice] A new release of pip available: 22.2.1 -> 23.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.1 -> 23.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Found model file at  ./wizardlm-13b-v1.1-superhot-8k.ggmlv3.q4_0.bin


objc[33739]: Class GGMLMetalClass is implemented in both /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x126d75228) and /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x127022228). One of the two will be used. Which one is undefined.
llama.cpp: loading model from ./wizardlm-13b-v1.1-superhot-8k.ggmlv3.q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32001
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rn

In [5]:
# Store in a vector database
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

In [6]:
# Enter a question and search based on similarity
question = "What is Aion?"
embedding_vector = embeddings.embed_query(question)
docs = vectorstore.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

Name: Aion
Description: Aion interest group is an officially recognized interest group for an Asian Interest Sorority at the University of California, Berkeley. We are striving to bring this Asian-Interest sorority to campus. We want our group to foster long-lasting relationships and build our community through breast cancer and mental health awareness, leadership, scholarship, and Asian awareness.


In [7]:
# Enter a question and get result
from langchain.chains import RetrievalQA
question = "What is Aion?"
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())
qa_chain({"query": question})

 Based on the provided contexts, it appears that "Aion" may be a term used within the Asian Interest Sorority at the University of California, Berkeley. However, without more information or clarification, we cannot definitively identify what Aion specifically refers to in this context.

{'query': 'What is Aion?',
 'result': ' Based on the provided contexts, it appears that "Aion" may be a term used within the Asian Interest Sorority at the University of California, Berkeley. However, without more information or clarification, we cannot definitively identify what Aion specifically refers to in this context.'}